### <span style='color:Blue'> L-wall design (BS8110). </span>
##### Based on "Design of structural elements by Chanakya Arya" page 125
##### assumption: No GWT, Passive Soil Neglected
##### Basic Python Programming Untuk Teknik Sipil - Merdeka Sandi Tazakka
---

In [1]:
import handcalcs.render
import numpy as np
import pandas as pd
from math import sqrt

from math import sin,radians,sqrt
import forallpeople as si
si.environment('structural', top_level=True)

#### Soil parameters

In [2]:
%%render params 1
phi = 30
gamma_soil = 19 *1000*N_m3
mu = 0.4
sigma_all = 120*kN/m**2

<IPython.core.display.Latex object>

#### Wall parameter

In [3]:
%%render params
gamma_c = 24 *1000*N_m3
H_wall = 5*m
H_slab = 400*mm
H_total = H_wall + H_slab
width_toe = 700*mm
width_wall = 400*mm
width_heel = 2900*mm
width_total = width_toe + width_wall + width_heel

<IPython.core.display.Latex object>

In [4]:
#untuk grafik saja
H_passive = 1000 #mm

#### Perhitungan Ka dan Kp

In [5]:
%%render 2
k_a =( 1 - sin(radians(phi))) / (1 + sin(radians(phi)))

k_p = ( 1 + sin(radians(phi))) / (1 - sin(radians(phi)))

<IPython.core.display.Latex object>

#### Horizontal driving pressure

In [6]:
%%render long
sigma_h = k_a * gamma_soil * H_total

<IPython.core.display.Latex object>

#### Horizontal driving force

In [7]:
%%render
F_A = 0.5 * sigma_h * H_total * (1*m)

<IPython.core.display.Latex object>

#### Weight calculation

In [8]:
%%render long
W_wall = width_wall * H_wall * gamma_c * (1*m)
W_base = H_slab * (width_heel+width_wall+width_toe) * gamma_c * (1*m)

<IPython.core.display.Latex object>

In [9]:
%%render long
W_soil = H_wall * (width_heel)*gamma_soil * (1*m)
W_total = W_base+W_soil+W_wall

<IPython.core.display.Latex object>

#### Factor of Safety Sliding

In [10]:
%%render long
F_friction = mu * W_total

<IPython.core.display.Latex object>

In [11]:
%%render
SF_sliding = F_friction / F_A

<IPython.core.display.Latex object>

In [12]:
if SF_sliding >= 1.5:
    print(f'SF sliding is {round(SF_sliding,2)} >= 1.5, OK')
else:
    print(f'SF sliding is {round(SF_sliding,2)} < 1.5, Not OK')

SF sliding is 1.57 >= 1.5, OK


#### Factor of safety overturning

In [13]:
%%render long
M_A = (F_A * H_total )/ 3

<IPython.core.display.Latex object>

In [14]:
%%render long
Arm_wall = (width_wall/2 + width_toe)
Arm_base = (width_total/2)
Arm_soil = (width_toe+width_wall+width_heel/2)
M_P = W_wall * Arm_wall + W_base * Arm_base  + W_soil * Arm_soil

<IPython.core.display.Latex object>

In [15]:
%%render
SF_overturning = M_P / M_A

<IPython.core.display.Latex object>

In [16]:
if SF_overturning >= 2:
    print(f'SF overturning is {round(SF_overturning,2)} >= 2, OK')
else:
    print(f'SF overturning is {round(SF_overturning,2)} < 2, Not OK')

SF overturning is 4.95 >= 2, OK


#### Bearing Capacity Check

In [17]:
%%render long
ecc_wall = width_total/2-width_toe-width_wall/2

<IPython.core.display.Latex object>

In [18]:

%%render long
ecc_soil = width_total/2-width_heel/2

<IPython.core.display.Latex object>

In [19]:
%%render long
N = W_total
M = (F_A * H_total) / 3 + W_wall * ecc_wall-W_soil * ecc_soil

check = M/N < width_total/6

<IPython.core.display.Latex object>

In [20]:
if check is True:
    print ('M/N < D/6, OK')
else:
    print ('M/N > D/6, check bearing pressure shape')

M/N < D/6, OK


In [21]:
%%render long
sigma_toe = N/(width_total*m) + (6*M)/(width_total**2*m)

<IPython.core.display.Latex object>

In [22]:
%%render long
sigma_heel = N/(width_total*m) - (6*M)/(width_total**2*m)

<IPython.core.display.Latex object>

In [23]:
%%render long
bearing = sigma_toe <= sigma_all

<IPython.core.display.Latex object>

In [24]:
if bearing is True:
    print ('bearing capacity OK')
else:
    print ('bearing capacity Not OK')

bearing capacity OK


### Reinforcement Calculation
#### Structural parameter

In [25]:
%%render params 2
gamma_f = 1.4
steel_dia = 20*mm
f_cu = 35 *MPa
f_y = 500 *MPa
cover_wall = 35*mm
cover_base = 50*mm

<IPython.core.display.Latex object>

#### Wall reinforcement per-meter run . Near Face / NF (tension side)

In [26]:
%%render long

F_soil = 0.5 * k_a * gamma_soil * H_wall**2 *(1*m)
M_wall = (gamma_f*F_soil*H_wall)/3

<IPython.core.display.Latex object>

In [27]:
%%render long
d_wall = width_wall - cover_wall - steel_dia/2
M_u = 0.156 * f_cu * d_wall**2 * (1*m)

<IPython.core.display.Latex object>

In [28]:
%%render long
check_M = M_wall <= M_u

<IPython.core.display.Latex object>

In [29]:
if bearing is True:
    print ('No need compression steel')
else:
    print ('Need compression steel, update all sheet')

No need compression steel


In [30]:
%%render long
K_wall = M_wall / (f_cu*1*m*d_wall**2)
z_wall = d_wall * (0.5 + sqrt(0.25 - K_wall/0.9))
As_wall_NF= M_wall / (0.87 * f_y * z_wall)

<IPython.core.display.Latex object>

#### Wall reinforcement per-meter run. Front Face / FF (compression side)

In [31]:
%%render long
steel_min = 0.13/100*(1*m)*width_wall
As_wall_FF = steel_min


<IPython.core.display.Latex object>

#### Base reinforcement ber-meter run (Heel)

In [32]:
%%render
p_1 = gamma_f * sigma_toe
p_2 = gamma_f * sigma_heel
p_3 = p_2 + (width_heel*(p_1-p_2))/width_total

<IPython.core.display.Latex object>

In [33]:
%%render long
M_soil = (W_soil*gamma_f*width_heel)/2
M_concrete = (width_heel*W_base*gamma_f)/width_total*(width_heel/2)

<IPython.core.display.Latex object>

In [34]:
%%render long
M_bearing1= (p_2*width_heel*width_heel*1*m)/2
M_bearing2 = (((p_3-p_2)*width_heel)/2*(width_heel*1/3)*(1*m))

<IPython.core.display.Latex object>

In [35]:
%%render long
M_heel = M_soil + M_concrete - M_bearing1 - M_bearing2


<IPython.core.display.Latex object>

In [36]:
%%render long 4
d_heel = H_slab - cover_base - steel_dia/2
K_heel = M_heel / (f_cu*1*m*d_heel**2)

<IPython.core.display.Latex object>

In [37]:
%%render long 4
z_heel = d_heel * (0.5 + sqrt(0.25 - K_heel/0.9))
As_heel= M_heel / (0.87 * f_y * z_heel)

<IPython.core.display.Latex object>

#### Base reinforcement ber-meter run (Toe)

In [38]:
%%render long
M_toe = (p_1 * width_toe**2)/2*(1*m) - (width_toe*W_base*gamma_f/width_total)*(width_toe/2)
As_toe = max((M_toe * As_heel)/M_heel, steel_min)


<IPython.core.display.Latex object>

#### Reinforcement summary

In [39]:
import pandas as pd

#units perlu dihilangkan
As1 = As_wall_NF / mm**2
As2 = As_wall_FF / mm**2
As3 = As_heel / mm**2
As4 = As_toe / mm**2
As_dist = steel_min /mm**2
steel_NF = steel_dia / mm
steel_heel = steel_dia / mm

#masukkan diameter steel yang lain
steel_FF = 12
steel_toe = 12
steel_dist = 12

df_recap = pd.DataFrame({
    'structure_part':['As_wall_NF','As_wall_FF','As_heel','As_toe','As_distribution'],
    'steel_diameter (mm)':[steel_NF,steel_FF,steel_heel,steel_toe,steel_dist],
    'As_required (mm2/m)':[As1,As2,As3,As4,As_dist]
})

#function untuk menghitung jumlah baja
def steel_n(x):
    hitung = True
    n = 1
    while hitung == True:
        kebutuhan_baja = x['As_required (mm2/m)']
        luas_perbatang = np.pi* x['steel_diameter (mm)']**2 /4
        luas_baja_terpasang = n * luas_perbatang

        if luas_baja_terpasang >= kebutuhan_baja:
            hitung = False
        else:
            n += 1
    return n

df_recap = df_recap.assign(n=df_recap.apply(steel_n, axis=1))

#menghitung As total dari hasil automatic calculation
df_recap['As_calculated'] = df_recap['steel_diameter (mm)']**2 / 4 * np.pi * df_recap['n']

#calculate spacing
kelipatan = 25
df_recap['spacing (mm)'] = kelipatan *  (1000 / df_recap['n']/kelipatan).apply(np.floor)

#create anotation
for i in range (len(df_recap)):
    df_recap.loc[i, 'annotation'] = f"D{df_recap['steel_diameter (mm)'][i]} @ {df_recap['spacing (mm)'][i]} mm"

df_recap

,structure_part,steel_diameter (mm),As_required (mm2/m),n,As_calculated,spacing (mm),annotation
0,As_wall_NF,20.0,1257.729100,5,1570.796327,200.0,D20.0 @ 200.0 mm
1,As_wall_FF,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm
2,As_heel,20.0,1135.015049,4,1256.637061,250.0,D20.0 @ 250.0 mm
3,As_toe,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm
4,As_distribution,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm


In [40]:
#RUBAH MANUAL n
df_recap.loc[2, 'n'] = 5


#menghitung As total dari hasil automatic calculation
df_recap['As_calculated'] = df_recap['steel_diameter (mm)']**2 / 4 * np.pi * df_recap['n']

#calculate spacing
kelipatan = 25
df_recap['spacing (mm)'] = kelipatan *  (1000 / df_recap['n']/kelipatan).apply(np.floor)

#create anotation
for i in range (len(df_recap)):
    df_recap.loc[i, 'annotation'] = f"D{df_recap['steel_diameter (mm)'][i]} @ {df_recap['spacing (mm)'][i]} mm"

df_recap

,structure_part,steel_diameter (mm),As_required (mm2/m),n,As_calculated,spacing (mm),annotation
0,As_wall_NF,20.0,1257.729100,5,1570.796327,200.0,D20.0 @ 200.0 mm
1,As_wall_FF,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm
2,As_heel,20.0,1135.015049,5,1570.796327,200.0,D20.0 @ 200.0 mm
3,As_toe,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm
4,As_distribution,12.0,520.000000,5,565.486678,200.0,D12.0 @ 200.0 mm


In [41]:
#menghilangkan satuan untuk plot grafik
H_wall = H_wall/m*1000
H_slab = H_slab/mm
H_total = H_wall + H_slab
width_toe = width_toe/mm
width_wall = width_wall/mm
width_heel = width_heel/mm
width_total = width_toe + width_wall + width_heel
cover_base = cover_base/mm
cover_wall = cover_wall/mm

In [42]:
#Major reinforcement
#As toe
x_rein_slab_bot= width_total-cover_base
y_rein_slab_bot= cover_base+steel_dist/2

#As heel
x_rein_slab_top= width_total-cover_base
y_rein_slab_top= H_slab - cover_base - steel_heel/2

#As FF
x_rein_wall_left = cover_wall+width_toe
y_rein_wall_left = H_total-cover_wall

#As NF
x_rein_wall_right = width_wall + width_toe - cover_wall
y_rein_wall_right = H_total-cover_wall

In [43]:
#Distribution reinforcment slab bot
x_dist_slab_bot = np.linspace(cover_base+df_recap['steel_diameter (mm)'][3],x_rein_slab_bot-df_recap['steel_diameter (mm)'][3],int(x_rein_slab_bot/df_recap['spacing (mm)'][3]+2))
y_dist_slab_bot = y_rein_slab_bot+50
dist_slab_bot = pd.DataFrame({'x':x_dist_slab_bot,'y':y_dist_slab_bot,})

#Distribution reinforcment slab top
x_dist_slab_top = np.linspace(cover_base+df_recap['steel_diameter (mm)'][2],x_rein_slab_top-df_recap['steel_diameter (mm)'][2],int(x_rein_slab_top/df_recap['spacing (mm)'][2]+2))

y_dist_slab_top = y_rein_slab_top-50
dist_slab_top = pd.DataFrame({'x':x_dist_slab_top,'y':y_dist_slab_top,})

#Distribution reinforcment NF
x_dist_wall_left = x_rein_wall_left+50
y_dist_wall_left = np.linspace(H_slab+df_recap['spacing (mm)'][1]+df_recap['steel_diameter (mm)'][1],y_rein_wall_left-df_recap['steel_diameter (mm)'][1],int(y_rein_wall_left/df_recap['spacing (mm)'][1]+2))

dist_wall_left  = pd.DataFrame({'x':x_dist_wall_left,'y':y_dist_wall_left,})
#Distribution reinforcment FF
x_dist_wall_right = x_rein_wall_right-50
y_dist_wall_right = np.linspace(H_slab+df_recap['spacing (mm)'][0]+df_recap['steel_diameter (mm)'][0],y_rein_wall_right-df_recap['steel_diameter (mm)'][0],int(y_rein_wall_right/df_recap['spacing (mm)'][0]+2))

dist_wall_right  = pd.DataFrame({'x':x_dist_wall_right,'y':y_dist_wall_right,})

In [44]:
#starter bar
starterbar_diameter = df_recap['steel_diameter (mm)'].iloc[0]
starterbar_spacing = df_recap['spacing (mm)'].iloc[0]
overlap_length = 40 * starterbar_diameter
bend_length = 16*starterbar_diameter
starterbar_annotation = f"D{starterbar_diameter} @ {starterbar_spacing} mm"

#### Plot grafik

In [46]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

offset = 100

fig = make_subplots(rows=1, cols=1)

#Soil surface
fig.add_shape(type="line",x0=-offset, y0=H_passive, x1=width_toe, y1=H_passive,line=dict(color="green",width=1), row=1, col=1)
fig.add_shape(type="line",x0=width_toe+width_wall, y0=H_total, x1=H_total+offset, y1=H_total,line=dict(color="green",width=1), row=1, col=1)

#concrete geometry
fig.add_shape(type="rect",
    x0=0, y0=0, x1=width_total, y1=H_slab,
    line=dict(
        color="RoyalBlue",
        width=0.2,
    ), row=1, col=1
)

fig.add_shape(type="rect",
    x0=width_toe+width_wall, y0=H_slab, x1=width_toe, y1=H_total,
    line=dict(
        color="RoyalBlue",
        width=0.2,
    ), row=1, col=1
)

#pembesian
#transversal
fig.append_trace(go.Scatter(x=[x_rein_wall_left,x_rein_wall_left,x_rein_wall_left+width_wall/2],
                         y=[cover_base+H_slab, y_rein_wall_left, y_rein_wall_left],
                         name=f"{df_recap['annotation'][0]}", mode='lines', legendgroup = '2'), row=1, col=1)

fig.append_trace(go.Scatter(x=[x_rein_wall_right,x_rein_wall_right,x_rein_wall_right-width_wall/2],
                         y=[cover_base+H_slab, y_rein_wall_right, y_rein_wall_right],
                         name=f"{df_recap['annotation'][1]}", mode='lines', legendgroup = '2'), row=1, col=1)

fig.append_trace(go.Scatter(x=[cover_base,cover_base,x_rein_slab_bot,x_rein_slab_bot],
                         y=[y_rein_slab_top-H_slab/2,y_rein_slab_top,y_rein_slab_top,y_rein_slab_top-H_slab/2],
                         name=f"{df_recap['annotation'][2]}", mode='lines', legendgroup = '2'), row=1, col=1)

fig.append_trace(go.Scatter(x=[cover_base,cover_base,x_rein_slab_bot,x_rein_slab_bot],
                         y=[y_rein_slab_bot+H_slab/2, y_rein_slab_bot,y_rein_slab_bot,y_rein_slab_bot+H_slab/2],
                         name=f"{df_recap['annotation'][3]}", mode='lines', legendgroup = '2'), row=1, col=1)

#longitudinal
fig.append_trace(go.Scatter(x=dist_slab_bot['x'],
                         y=dist_slab_bot['y'],
                         name=f"{df_recap['annotation'][3]}", mode='markers',marker_size=4, legendgroup = '3'), row=1, col=1)

fig.append_trace(go.Scatter(x=dist_slab_top['x'],
                         y=dist_slab_top['y'],
                         name=f"{df_recap['annotation'][2]}", mode='markers',marker_size=4, legendgroup = '3'), row=1, col=1)

fig.append_trace(go.Scatter(x=dist_wall_left['x'],
                         y=dist_wall_left['y'],
                         name=f"{df_recap['annotation'][1]}", mode='markers',marker_size=4, legendgroup = '3'), row=1, col=1)

fig.append_trace(go.Scatter(x=dist_wall_right['x'],
                         y=dist_wall_right['y'],
                         name=f"{df_recap['annotation'][1]}", mode='markers',marker_size=4, legendgroup = '3'), row=1, col=1)

#starter bar
fig.append_trace(go.Scatter(x=[x_rein_wall_left+100-bend_length,x_rein_wall_left+100,x_rein_wall_left+100],
                         y=[y_rein_slab_bot+100,y_rein_slab_bot+100,H_slab+overlap_length],
                         name=starterbar_annotation, mode='lines',marker_size=4, legendgroup = '4'), row=1, col=1)

fig.append_trace(go.Scatter(x=[x_rein_wall_right-100+bend_length,x_rein_wall_right-100,x_rein_wall_right-100],
                         y=[y_rein_slab_bot+100,y_rein_slab_bot+100,H_slab+overlap_length],
                         name=starterbar_annotation, mode='lines',marker_size=4, legendgroup = '4'), row=1, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="x (mm)", row=1, col=1, side ="bottom",range=[-offset,H_total+offset], tickmode = 'linear', tick0 = 0, dtick = 1000 , linecolor='black', gridcolor='white' )

# Update yaxis properties
fig.update_yaxes(title_text="y (mm)", row=1, col=1, range=[-offset,H_total+offset], tickmode = 'linear', tick0 = 0, dtick = 1000, linecolor='black', gridcolor='white')

fig.update_layout(title_text="", width = 1000, height=900, legend_tracegroupgap=20, plot_bgcolor='rgb(245,245,240)', bargap=0,bargroupgap=0)
fig.show()